In [1]:
!pip install unsloth transformers trl

In [2]:
#transformer for atttention layers
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth.chat_templates import get_chat_template, standardize_sharegpt


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 2048,
    load_in_4bit = True
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]
)
#https://www.youtube.com/redirect?event=video_description&redir_token=QUFFLUhqbmxHS2d2NVdnbnc2cFRvMDNOanVKSjJxR3drZ3xBQ3Jtc0trRXJNbmhONGMxVUt2aFlnRklJZFBKenB0ZHNUQUY4T2lGa3pveVhWTzdrcW9lbjlpdXlnVldPZEQ4UFF5dTd6c19KSmJleEpFbkNBZWI4Z3VYODU3OVlWbU5rVUdrUlRIQkJYVlk0U3EwbWtTMXN6MA&q=https%3A%2F%2Fmedium.com%2F%40kalra.rakshit%2Fintroduction-to-transformers-and-attention-mechanisms-c29d252ea2c5&v=dn2anUU0d0U

Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [7]:
tokenizer = get_chat_template(tokenizer , chat_template="llama-3.1")

In [9]:
dataset = load_dataset("mlabonne/FineTome-100k",split="train")

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [10]:
dataset = standardize_sharegpt(dataset)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [11]:
dataset[0]

{'conversations': [{'content': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.',
   'role': 'user'},
  {'content': 

In [14]:
dataset = dataset.map(
    lambda examples:{
        "text":[
            tokenizer.apply_chat_template(convo,tokenize = False)
            for convo in examples["conversations"]
        ]
    },
    batched = True
)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [15]:
dataset[0]

{'conversations': [{'content': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.',
   'role': 'user'},
  {'content': 

In [16]:
trainer = SFTTrainer(
    model = model,
    train_dataset= dataset,
    dataset_text_field="text",
    max_seq_length = 2048 ,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported() ,
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        output_dir = "outputs"
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [17]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tt1221351 (tt1221351-iit-delhi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.492200
2,2.629700
3,1.621900
4,1.940100
5,1.900500
6,1.977800
7,1.147100
8,1.859500
9,1.525300
10,1.451900


TrainOutput(global_step=60, training_loss=1.1826811263958612, metrics={'train_runtime': 637.744, 'train_samples_per_second': 0.753, 'train_steps_per_second': 0.094, 'total_flos': 6240291994460160.0, 'train_loss': 1.1826811263958612})

In [18]:
model.save_pretrained("finetuned_model")

In [19]:
inference_model , inference_tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./finetuned_model",
    max_seq_length = 2048,
    load_in_4bit = True
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [23]:
text_prompts = [
    "what are the key principles of investment?"
]

for prompt in text_prompts:
    formatted_prompt = inference_tokenizer.apply_chat_template([{
        "role":"user",
        "content":prompt
    }],tokenize = False)
    model_inputs = inference_tokenizer(formatted_prompt,return_tensors="pt").to("cuda")
    generated_ids = inference_model.generate(
        **model_inputs,
        max_new_tokens = 512,
        temperature =0.7,
        do_sample = True ,
        pad_token_id = inference_tokenizer.eos_token_id
    )
    response = inference_tokenizer.batch_decode(generated_ids,skip_special_tokens=True)[0]
    print(response)

system

Cutting Knowledge Date: December 2023
Today Date: 23 Apr 2025

user

what are the key principles of investment?assistant

The key principles of investment are based on the idea of generating returns on investment (ROI) while managing risk. These principles are often used by investors, financial advisors, and portfolio managers to make informed investment decisions. Here are some of the key principles of investment:

1. **Diversification**: Spread investments across different asset classes, sectors, and geographies to minimize risk and maximize potential returns.
2. **Risk management**: Assess and manage risk by diversifying investments, hedging against potential losses, and setting stop-loss orders.
3. **Time horizon**: Consider the investor's time horizon when making investment decisions. Short-term investments may be riskier, while long-term investments can be more stable.
4. **Dollar-cost averaging**: Invest a fixed amount of money at regular intervals, regardless of market 